
🎥 Recommendation Video: [Cosine Similarity, Clearly Explained!!!](https://www.youtube.com/watch?v=e9U0QAFbfLI)


# Introduction to Recommendation Systems: Guiding Users to What They Love

You have browsed an e-commerce site before, and it suggests a pair of shoes that perfectly matches your taste. Or stream a movie platform where every recommendation aligns with your viewing preferences. These experiences are powered by **Recommendation Systems**, a class of machine learning algorithms designed to predict and suggest relevant options to users.

Recommendation systems are the backbone of platforms like music streaming, video services, and online shopping. Their goal? To connect users with content they’ll love by quantifying similarities between items or users.

---

## Types of Recommendation Systems

Recommendation systems primarily fall into two categories:

### 1. Content-Based Filtering

This approach recommends items by analyzing their attributes and comparing them to what the user has liked in the past. It focuses on the **features of the items** rather than user preferences.

#### How It Works:
- The system evaluates the characteristics of items (e.g., genre, artist, tempo for songs).
- It compares these features with items the user has already interacted with.
- Recommendations are based on similarity scores between items.

#### Example:
On a music streaming service:
- A user enjoys fast-tempo, electronic songs by Artist X.
- The system recommends other fast-tempo, electronic songs, even from different artists, because they share similar attributes.

---

### 2. Collaborative Filtering

Collaborative filtering leverages the preferences of other users to make recommendations. It assumes that users with similar tastes will like similar items, even without analyzing the content itself.

#### How It Works:
- The system identifies users with similar interaction patterns (e.g., ratings, likes).
- It recommends items liked by these similar users that the current user hasn’t tried yet.

#### Example:
On an e-commerce platform:
- User A and User B have both liked Products 1, 2, and 3.
- User A buys Product 4.
- The system recommends Product 4 to User B, assuming similar preferences.

---

Recommendation systems bridge the gap between users and vast amounts of available content, helping platforms personalize experiences and keep users engaged. Whether through analyzing content attributes or leveraging collective preferences, these systems ensure users discover what they value most.



## Example: A simple Movie Recommendation System

In [ ]:
# Importing necessary libraries
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
# Sample movie data with genres
movies = pd.DataFrame({
    'Title': ['Movie A', 'Movie B', 'Movie C', 'Movie D'],
    'Genre': ['Action|Adventure', 'Action|Thriller', 'Comedy|Romance', 'Action|Comedy']
})

# User ratings data for the movies
ratings = pd.DataFrame({
    'User': ['Alice', 'Alice', 'Bob', 'Bob', 'Charlie', 'Charlie'],
    'Title': ['Movie A', 'Movie B', 'Movie A', 'Movie C', 'Movie B', 'Movie C'],
    'Rating': [5, 4, 4, 5, 5, 4]
})

# Displaying the data
print("Movies Dataset:")
display(movies)
print("\nRatings Dataset:")
display(ratings)


Movies Dataset:


,Title,Genre
0,Movie A,Action|Adventure
1,Movie B,Action|Thriller
2,Movie C,Comedy|Romance
3,Movie D,Action|Comedy



Ratings Dataset:


,User,Title,Rating
0,Alice,Movie A,5
1,Alice,Movie B,4
2,Bob,Movie A,4
3,Bob,Movie C,5
4,Charlie,Movie B,5
5,Charlie,Movie C,4


### Content-Based Recommendations

In [ ]:
# Function to recommend movies based on content (genres)
def content_based_recommendations(movie_title, movies_df):
    """
    Recommends movies similar to the given movie based on genre.

    Args:
    movie_title (str): Title of the movie for which recommendations are made.
    movies_df (DataFrame): DataFrame containing movie titles and genres.

    Returns:
    list: List of recommended movie titles.
    """
    # Vectorizing the 'Genre' column using TF-IDF
    tfidf = TfidfVectorizer()
    tfidf_matrix = tfidf.fit_transform(movies_df['Genre'])

    # Calculating cosine similarity between all movies
    cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

    # Getting the index of the given movie
    indices = pd.Series(movies_df.index, index=movies_df['Title'])
    idx = indices[movie_title]

    # Getting similarity scores for the given movie
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sorting scores in descending order and selecting top 2 recommendations
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)[1:3]

    # Getting indices of recommended movies
    recommended_indices = [i[0] for i in sim_scores]

    # Returning the titles of the recommended movies
    return movies_df['Title'].iloc[recommended_indices]

# Testing the function
print("Content-Based Recommendations for 'Movie A':")
print(content_based_recommendations('Movie A', movies))


Content-Based Recommendations for 'Movie A':
3    Movie D
1    Movie B
Name: Title, dtype: object


### Collaborative Filtering Function

In [ ]:
# Function to recommend movies based on collaborative filtering
def collaborative_filtering(user_name, ratings_df):
    """
    Recommends movies to a user based on ratings from similar users.

    Args:
    user_name (str): Name of the user for whom recommendations are made.
    ratings_df (DataFrame): DataFrame containing user ratings.

    Returns:
    list: List of recommended movie titles.
    """
    # Creating a user-item matrix
    user_item_matrix = ratings_df.pivot_table(index='User', columns='Title', values='Rating')

    # Calculating user-user similarity using cosine similarity
    user_similarity = cosine_similarity(user_item_matrix.fillna(0))

    # Finding the index of the given user
    user_idx = list(user_item_matrix.index).index(user_name)

    # Getting similarity scores for the given user
    sim_scores = list(enumerate(user_similarity[user_idx]))

    # Sorting scores in descending order and selecting top 2 similar users
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)[1:3]

    # Getting the names of similar users
    similar_users = [user_item_matrix.index[i[0]] for i in sim_scores]

    # Finding movies liked by similar users but not rated by the given user
    recommendations = ratings_df[
        ratings_df['User'].isin(similar_users) &
        ~ratings_df['Title'].isin(ratings_df[ratings_df['User'] == user_name]['Title'])
    ]

    # Returning the titles of the recommended movies
    return recommendations['Title'].unique()

# Testing the function
print("Collaborative Filtering Recommendations for 'Alice':")
print(collaborative_filtering('Alice', ratings))


Collaborative Filtering Recommendations for 'Alice':
['Movie C']
